In [1]:
import json

# load .json
data_path = "/home/liyi/workspace/dataset/vqa/rlbench_all_tasks_256_1000_eps_sketch_v5_rlbench.json"
data = json.load(open(data_path))

In [12]:
import os
from tqdm import tqdm
from vila_utils.utils.prompts import get_prompt

task = "rlbench_hamster"
data_dir = f"/lustre/fs12/portfolios/nvr/users/mmemmel/projects/vila/data/hamster/{task}"
os.makedirs(data_dir, exist_ok=True)

entries_conv = []
entries_vqa = []

prompt_type = "path"
split = "train"

for sample in tqdm(data):
    
    quest = sample["conversations"][0]["value"]
    quest = quest.split("<quest>")[1].split("</quest>")[0]
    
    answer = sample["conversations"][1]["value"]
    # remove gripper from answer
    answer = answer.replace("<action>Close Gripper</action>", "")
    answer = answer.replace("<action>Open Gripper</action>", "")
    answer = answer.replace(", ,", ",")
    answer = answer.replace(", ]", "]")
    answer = answer.replace("[, ", "[")
    
    # print(quest)
    # print(answer)
    
    # construct conversation format (train)
    entry_conv = {
        "image": sample["image"],
        "id":  sample["id"],
        "conversations": [
            {
                "from": "human",
                "value": get_prompt(quest, prompt_type=prompt_type, prompt_eval=False),
            },
            {
                "from": "gpt",
                "value": answer
            }
        ]
    }
    entries_conv.append(entry_conv)

    # constract VQA format (eval)
    entry_vqa = {
        "image": sample["image"],
        "question_id": sample["id"],
        # same as above but w/o <image>\n -> https://github.com/haotian-liu/LLaVA/issues/732#issuecomment-1994616417
        "text": get_prompt(quest, prompt_type=prompt_type, prompt_eval=True),
        "category": task
    }
    entries_vqa.append(entry_vqa)

vqa_file = f'{data_dir}/{split}_{task}_vqa.jsonl'
if os.path.exists(vqa_file):
    os.remove(vqa_file)

with open(vqa_file, 'a') as file:
    for entry_vqa in entries_vqa:
        file.write(json.dumps(entry_vqa) + '\n')

conv_file = f'{data_dir}/{split}_{task}_conv.json'
if os.path.exists(conv_file):
    os.remove(conv_file)
with open(conv_file, 'w') as file:
    json.dump(entries_conv, file)

print(conv_file)

100%|██████████| 366332/366332 [00:01<00:00, 185899.46it/s]


/lustre/fs12/portfolios/nvr/users/mmemmel/projects/vila/data/hamster/rlbench_hamster/train_rlbench_hamster_conv.json
